# Notebook

In [3]:
print(potato(2, 4))

6


In [4]:
# installs- comment out if good

!git clone https://github.com/wtlu71/cnn-comp-med.git

import numpy as np
import os
from tqdm import tqdm
import sys
import os

repo_path = '/content/cnn-comp-med'
# Add to Python path
sys.path.append(repo_path)
# imports
from my_scripts.test import potato
from my_scripts.my_models import SmallCNN, SmallMLP
from my_scripts.dataset_loading import H5Dataset
from my_scripts.utils import run_epoch

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, Subset
import torchvision.models as models
from torchvision.models import resnet50
from torchvision import datasets, transforms

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
# Path to your repo in Drive

fatal: destination path 'cnn-comp-med' already exists and is not an empty directory.


Get train/val/test data from Zenodo:

In [5]:
os.makedirs(os.path.join(os.getcwd(),"data"),exist_ok=True)

In [6]:
# train

!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_x.h5.gz?download=1 -O data/train_x.h5.gz
!gunzip data/train_x.h5.gz

!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_y.h5.gz?download=1 -O data/train_y.h5.gz
!gunzip data/train_y.h5.gz

# val
!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz?download=1 -O data/valid_x.h5.gz
!gunzip data/valid_x.h5.gz

!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_valid_y.h5.gz?download=1 -O data/valid_y.h5.gz
!gunzip data/valid_y.h5.gz

# test
!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_test_x.h5.gz?download=1 -O data/test_x.h5.gz
!gunzip data/test_x.h5.gz

!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_test_y.h5.gz?download=1 -O data/test_y.h5.gz
!gunzip data/test_y.h5.gz

--2025-11-26 22:39:43--  https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_x.h5.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.52.235, 188.185.43.153, 188.185.48.75, ...
Connecting to zenodo.org (zenodo.org)|137.138.52.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6421353462 (6.0G) [application/octet-stream]
Saving to: ‘data/train_x.h5.gz’

data/train_x.h5.gz  100%[===================>]   5.98G  17.2MB/s    in 5m 48s  

2025-11-26 22:45:32 (17.6 MB/s) - ‘data/train_x.h5.gz’ saved [6421353462/6421353462]

gzip: invalid option -- 'y'
Try `gzip --help' for more information.
--2025-11-26 22:45:33--  https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_y.h5.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.52.235, 188.185.48.75, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|137.138.52.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21378 (21K) [applic

In [7]:
# define transforms

# is there a way to get this programmatically from the data or not worth it
IMG_SIZE = 96
BATCH_SIZE = 3

# Training transforms with augmentation
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(10),
    # TODO: normalize: mean and std for ImageNet (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation/Test transforms (no augmentation)
eval_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    # TODO: normalize: mean and std for ImageNet (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Data transforms defined")

Data transforms defined


In [8]:
# dataset paths- Colab virtual session
train_img_h5_path = "data/train_x.h5"
train_label_h5_path = "data/train_y.h5"

val_img_h5_path = "data/valid_x.h5"
val_label_h5_path = "data/valid_y.h5"

test_img_h5_path = "data/test_x.h5"
test_label_h5_path = "data/test_y.h5"

train_subset_size = 50
eval_subset_size = 10

train_dataset = H5Dataset(train_img_h5_path,train_label_h5_path,transform=train_transforms)
train_dataset_subset = Subset(train_dataset, range(train_subset_size))
train_loader = DataLoader(train_dataset_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

val_dataset = H5Dataset(val_img_h5_path,val_label_h5_path,transform=train_transforms)
val_dataset_subset = Subset(val_dataset, range(eval_subset_size))
val_loader = DataLoader(val_dataset_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = H5Dataset(test_img_h5_path,test_label_h5_path,transform=train_transforms)
test_dataset_subset = Subset(test_dataset, range(eval_subset_size))
test_loader = DataLoader(test_dataset_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [9]:
# define/load models
mlpmodel = SmallMLP().to(device)
smallcnnmodel = SmallCNN().to(device)
resnetmodel = resnet50(weights = models.ResNet50_Weights.IMAGENET1K_V2).to(device)

# training hyperparameters
# can change learning rate or use scheduler
LEARNING_RATE = 3e-4
# finetune with less epochs to avoid forgetting
EPOCHS = 10
PATIENCE = 5
# patience- number of epochs the model continues after no improvement in validation loss

# consider other loss functions https://neptune.ai/blog/pytorch-loss-functions
criterion = nn.CrossEntropyLoss()
# don't change adam, never change dude
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 191MB/s]


In [10]:
modellist = [mlpmodel, smallcnnmodel, resnetmodel]
# modellist = [mlpmodel, smallcnnmodel]

historylist = []
stopepochs = []
for _, model in enumerate(modellist):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # try to integrate into wandb instead of storing this so we can have a pretty dashboard?
    history = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": [],
        "val_auc": []
    }

    best_auc = -np.inf
    best_state = None
    bad_epochs = 0

    print("\nStarting training...\n")
    for epoch in tqdm(range(1, EPOCHS + 1)):
        # TODO: Train for one epoch
        tr_loss, tr_acc, _, _, _ = run_epoch(train_loader, model, criterion, optimizer=optimizer, train=True)

        # TODO: Validate
        va_loss, va_acc, va_sens, va_spec, va_auc = run_epoch(val_loader, model, criterion, optimizer=None, train=False,device=device)

        # TODO: Store metrics
        history["train_loss"].append(tr_loss)
        history["val_loss"].append(va_loss)
        history["train_acc"].append(tr_acc)
        history["val_acc"].append(va_acc)
        history["val_auc"].append(va_auc)

        print(f"Epoch {epoch:02d}: "
            f"train_loss={tr_loss:.4f} "
            f"val_loss={va_loss:.4f} "
            f"val_acc={va_acc:.3f} "
            f"val_auc={va_auc:.3f}")

        # TODO: Early stopping logic
        if va_auc > best_auc + 1e-4:
            # TODO: Update the best AUC
            best_auc = va_auc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= PATIENCE:
                print(f"\nEarly stopping at epoch {epoch}")
                break

    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"\nRestored best model (val_auc={best_auc:.4f})")

    historylist.append(history)
    stopepochs.append(epoch)


Starting training...



  0%|          | 0/10 [00:00<?, ?it/s]


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataset.py", line 416, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
            ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "/content/cnn-comp-med/my_scripts/dataset_loading.py", line 31, in __getitem__
    self._open_file()
  File "/content/cnn-comp-med/my_scripts/dataset_loading.py", line 20, in _open_file
    self.img_h5_file = h5py.File(self.img_

In [ ]:
# finally, evaluate on test set

for _, model in tqdm(enumerate(modellist)):
    _, va_acc, va_sens, va_spec, va_auc = run_epoch(test_loader, model, criterion, train=False)
    print(f"\nFinal Validation Performance:")
    print(f"  AUC:         {va_auc:.4f}")
    print(f"  Accuracy:    {va_acc:.4f}")
    print(f"  Sensitivity: {va_sens:.4f}")
    print(f"  Specificity: {va_spec:.4f}")